In [ ]:
#r "nuget:System.Text.Json, 8.0.3"
#r "nuget:Azure.AI.DocumentIntelligence, 1.0.0-beta.2"
#r "nuget:Azure.Identity, 1.12.0"
#r "nuget:DotNetEnv, 3.0.0"

In [ ]:
using System.Net;
using System.Net.Http;
using System.Text.Json.Nodes;
using System.Text.Json;
using System.IO; 

using Azure;
using Azure.AI.DocumentIntelligence;
using Azure.Identity;
using DotNetEnv;

In [ ]:
Env.Load("../../.env");

string documentIntelligenceEndpoint = Environment.GetEnvironmentVariable("DOCUMENT_INTELLIGENCE_ENDPOINT");

var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions { 
    ExcludeEnvironmentCredential = true,
    ExcludeManagedIdentityCredential = true,
    ExcludeSharedTokenCacheCredential = true,
    ExcludeInteractiveBrowserCredential = true,
    ExcludeAzurePowerShellCredential = true,
    ExcludeVisualStudioCodeCredential = false,
    ExcludeAzureCliCredential = false
});

var documentName = "../Assets/Invoices/Invoice-Complex.pdf";

var documentIntelligenceClient = new DocumentIntelligenceClient(new Uri(documentIntelligenceEndpoint), credential);


In [ ]:
AnalyzeDocumentContent markdownAnalysisContent = new AnalyzeDocumentContent()
{
    Base64Source = BinaryData.FromBytes(File.ReadAllBytes(documentName))
};

Operation<AnalyzeResult> markdownAnalysisOperation = await documentIntelligenceClient.AnalyzeDocumentAsync(
    WaitUntil.Completed, 
    "prebuilt-invoice", 
    markdownAnalysisContent, 
    features: new[] { DocumentAnalysisFeature.QueryFields },
    outputContentFormat: ContentFormat.Markdown, 
    queryFields: new List<string> { "DeliveryDate", "CollectionDate", "CustomerSignatoryName", "DriverSignatoryName"});

In [ ]:
public class CustomInvoice(IReadOnlyDictionary<string, DocumentField> fields) : Invoice(fields)
{
    public DateTimeOffset? DeliveryDate { get; set; }
    public DateTimeOffset? CollectionDate { get; set; }
    public string? CustomerSignatoryName { get; set; }
    public string? DriverSignatoryName { get; set; }

    public override void InitializeFromFields(IReadOnlyDictionary<string, DocumentField> fields)
    {
        base.InitializeFromFields(fields);

        foreach (var field in fields)
        {
            switch (field.Key)
            {
                case "DeliveryDate":
                    DeliveryDate = DateTimeOffset.TryParse(field.Value.Content, out var deliveryDate) ? deliveryDate : default(DateTimeOffset?);
                    break;
                case "CollectionDate":
                    CollectionDate = DateTimeOffset.TryParse(field.Value.Content, out var collectionDate) ? collectionDate : default(DateTimeOffset?);
                    break;
                case "CustomerSignatoryName":
                    CustomerSignatoryName = field.Value.Content;
                    break;
                case "DriverSignatoryName":
                    DriverSignatoryName = field.Value.Content;
                    break;
            }
        }
    }
}

public class Invoice
{
    public Invoice(IReadOnlyDictionary<string, DocumentField> fields)
    {
        InitializeFromFields(fields);
    }

    public string? CustomerName { get; set; }
    public string? CustomerId { get; set; }
    public string? PurchaseOrder { get; set; }
    public string? InvoiceId { get; set; }
    public DateTimeOffset? InvoiceDate { get; set; }
    public DateTimeOffset? DueDate { get; set; }
    public string? VendorName { get; set; }
    public string? VendorAddress { get; set; }
    public string? VendorAddressRecipient { get; set; }
    public string? CustomerAddress { get; set; }
    public string? CustomerAddressRecipient { get; set; }
    public string? BillingAddress { get; set; }
    public string? BillingAddressRecipient { get; set; }
    public string? ShippingAddress { get; set; }
    public string? ShippingAddressRecipient { get; set; }
    public double? SubTotal { get; set; }
    public double? TotalDiscount { get; set; }
    public double? InvoiceTotal { get; set; }
    public double? AmountDue { get; set; }
    public double? PreviousUnpaidBalance { get; set; }
    public string? ServiceAddress { get; set; }
    public string? ServiceAddressRecipient { get; set; }
    public DateTimeOffset? ServiceStartDate { get; set; }
    public DateTimeOffset? ServiceEndDate { get; set; }
    public string? VendorTaxId { get; set; }
    public string? CustomerTaxId { get; set; }
    public string? PaymentTerm { get; set; }
    public string? KVKNumber { get; set; }
    public string? CurrencyCode { get; set; }
    public IEnumerable<string>? PaymentDetails { get; set; }
    public IEnumerable<string>? TaxDetails { get; set; }
    public IEnumerable<InvoiceItem>? Items { get; set; }

    public class InvoiceItem
    {
        public double? Amount { get; set; }
        public DateTimeOffset? Date { get; set; }
        public string? Description { get; set; }
        public double? Quantity { get; set; }
        public string? ProductCode { get; set; }
        public double? Tax { get; set; }
        public string? TaxRate { get; set; }
        public string? Unit { get; set; }
        public double? UnitPrice { get; set; }
    }

    public virtual void InitializeFromFields(IReadOnlyDictionary<string, DocumentField> fields)
    {
        foreach (var field in fields)
        {
            switch (field.Key)
            {
                case "CustomerName":
                    CustomerName = field.Value.Content;
                    break;
                case "CustomerId":
                    CustomerId = field.Value.Content;
                    break;
                case "PurchaseOrder":
                    PurchaseOrder = field.Value.Content;
                    break;
                case "InvoiceId":
                    InvoiceId = field.Value.Content;
                    break;
                case "InvoiceDate":
                    InvoiceDate = field.Value.ValueDate;
                    break;
                case "DueDate":
                    DueDate = field.Value.ValueDate;
                    break;
                case "VendorName":
                    VendorName = field.Value.Content;
                    break;
                case "VendorAddress":
                    VendorAddress = field.Value.Content;
                    break;
                case "VendorAddressRecipient":
                    VendorAddressRecipient = field.Value.Content;
                    break;
                case "CustomerAddress":
                    CustomerAddress = field.Value.Content;
                    break;
                case "CustomerAddressRecipient":
                    CustomerAddressRecipient = field.Value.Content;
                    break;
                case "BillingAddress":
                    BillingAddress = field.Value.Content;
                    break;
                case "BillingAddressRecipient":
                    BillingAddressRecipient = field.Value.Content;
                    break;
                case "ShippingAddress":
                    ShippingAddress = field.Value.Content;
                    break;
                case "ShippingAddressRecipient":
                    ShippingAddressRecipient = field.Value.Content;
                    break;
                case "SubTotal":
                    SubTotal = field.Value.ValueCurrency.Amount;
                    break;
                case "TotalDiscount":
                    TotalDiscount = field.Value.ValueCurrency.Amount;
                    break;
                case "InvoiceTotal":
                    InvoiceTotal = field.Value.ValueCurrency.Amount;
                    break;
                case "AmountDue":
                    AmountDue = field.Value.ValueCurrency.Amount;
                    break;
                case "PreviousUnpaidBalance":
                    PreviousUnpaidBalance = field.Value.ValueCurrency.Amount;
                    break;
                case "ServiceAddress":
                    ServiceAddress = field.Value.Content;
                    break;
                case "ServiceAddressRecipient":
                    ServiceAddressRecipient = field.Value.Content;
                    break;
                case "ServiceStartDate":
                    ServiceStartDate = field.Value.ValueDate;
                    break;
                case "ServiceEndDate":
                    ServiceEndDate = field.Value.ValueDate;
                    break;
                case "VendorTaxId":
                    VendorTaxId = field.Value.Content;
                    break;
                case "CustomerTaxId":
                    CustomerTaxId = field.Value.Content;
                    break;
                case "PaymentTerm":
                    PaymentTerm = field.Value.Content;
                    break;
                case "KVKNumber":
                    KVKNumber = field.Value.Content;
                    break;
                case "CurrencyCode":
                    CurrencyCode = field.Value.Content;
                    break;
                case "PaymentDetails":
                    PaymentDetails = field.Value.ValueList.Select(x => x.Content);
                    break;
                case "TaxDetails": 
                    TaxDetails = field.Value.ValueList.Select(x => x.Content);
                    break;
                case "Items":
                    Items = field.Value.ValueList.Select(x => x.ValueDictionary).Select(x => new InvoiceItem
                    {
                        Amount = x.TryGetValue("Amount", out var amount) ? amount.ValueCurrency.Amount : default(double?),
                        Date = x.TryGetValue("Date", out var date) ? date.ValueDate : default(DateTimeOffset?),
                        Description = x.TryGetValue("Description", out var description) ? description.Content : default(string?),
                        Quantity = x.TryGetValue("Quantity", out var quantity) ? quantity.ValueDouble : default(int?),
                        ProductCode = x.TryGetValue("ProductCode", out var productCode) ? productCode.Content : default(string?),
                        Tax = x.TryGetValue("Tax", out var tax) ? tax.ValueCurrency.Amount : default(double?),
                        TaxRate = x.TryGetValue("TaxRate", out var taxRate) ? taxRate.Content : default(string?),
                        Unit = x.TryGetValue("Unit", out var unit) ? unit.Content : default(string?),
                        UnitPrice = x.TryGetValue("UnitPrice", out var unitPrice) ? unitPrice.ValueCurrency.Amount : default(double?)
                    });
                    break;
            }
        }
    }
}

In [ ]:
AnalyzeResult markdownAnalysisResult = markdownAnalysisOperation.Value;

var invoiceFields = markdownAnalysisResult.Documents[0].Fields;
var invoice = new CustomInvoice(markdownAnalysisResult.Documents[0].Fields);

File.WriteAllText("PrebuiltInvoiceExtraction-FieldsResponse.json", JsonSerializer.Serialize(invoiceFields, new JsonSerializerOptions { WriteIndented = true }));
File.WriteAllText("PrebuiltInvoiceExtraction-ContentResponse.json", JsonSerializer.Serialize(invoice, new JsonSerializerOptions { WriteIndented = true }));

Console.WriteLine("PrebuiltInvoiceExtraction-FieldsResponse.json has been created from the Azure AI Document Intelligence service response.");
Console.WriteLine("PrebuiltInvoiceExtraction-ContentResponse.json has been created from parsing the Azure AI Document Intelligence service response into a POCO.");